In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('C:/Users/jcrig/Documents/FORMATION_SORBONNE/Machine_Learning_2_Faik/grand_ml_project/input_data/rating.csv')

In [5]:
df.head(5)

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [8]:
df = df.drop(['timestamp'], axis=1)

In [9]:
df.head(5)

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


In [15]:
df.shape

(20000263, 3)

In [21]:
df.dtypes

userId       int64
movieId      int64
rating     float64
dtype: object

In [22]:
df['userId'] = df['userId'].astype('int32')
df['movieId'] = df['movieId'].astype('int32')
df['rating'] = df['rating'].astype('float32')


In [23]:
df.dtypes

userId       int32
movieId      int32
rating     float32
dtype: object

In [10]:
nan_counts = df.isna().sum()
nan_counts

userId     0
movieId    0
rating     0
dtype: int64

In [13]:
df['movieId'].nunique()

26744

In [16]:
df['userId'].nunique()

138493

In [25]:
user_splits = np.array_split(df['userId'].unique(), 10)

df_pivot_list = []
for split in user_splits:
    df_subset = df[df['userId'].isin(split)]
    df_pivot_subset = pd.pivot_table(df_subset, values='rating', index='userId', columns='movieId')
    df_pivot_list.append(df_pivot_subset)

df_pivot = pd.concat(df_pivot_list)


In [26]:
df_pivot.shape

(138493, 26744)

In [27]:
df_pivot.head(5)

movieId,1,2,3,4,5,6,7,8,9,10,...,130586,130604,130614,130622,130656,130828,131110,131172,131237,131262
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
df = pd.read_csv('C:/Users/jcrig/Documents/FORMATION_SORBONNE/Machine_Learning_2_Faik/grand_ml_project/input_data/movie.csv')

In [56]:
df.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [57]:
df.set_index('movieId', inplace = True)

In [58]:
df.head(5)

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [59]:
df['title'] = df['title'].str.replace(r'\(\d{4}\)', '', regex=True)
df['title'] = df['title'].str.strip()


In [60]:
df.head(5)

,title,genres
movieId,,
1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji,Adventure|Children|Fantasy
3,Grumpier Old Men,Comedy|Romance
4,Waiting to Exhale,Comedy|Drama|Romance
5,Father of the Bride Part II,Comedy


In [62]:
# One-hot encoding des genres
genres_encoded = df['genres'].str.get_dummies(sep='|')

# Concatenation avec le DataFrame original
df_encoded = pd.concat([df, genres_encoded], axis=1)
df_encoded.head(5)

,title,genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,,
1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Jumanji,Adventure|Children|Fantasy,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Grumpier Old Men,Comedy|Romance,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,Waiting to Exhale,Comedy|Drama|Romance,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
5,Father of the Bride Part II,Comedy,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
df_encoded = df_encoded.drop(['genres'], axis=1)
df_encoded.shape

(27278, 20)

In [65]:
df_encoded.head(5)

,title,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,
1,Toy Story,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,Jumanji,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,Grumpier Old Men,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,Waiting to Exhale,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
5,Father of the Bride Part II,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [61]:
def recommend(movie_title):
    print("For movie ({})".format(movie_title))
    print("- Top 10 movies recommended based on Pearsons'R correlation - ")
    i = int(df.index[df['title'] == movie_title][0])
    target = df_pivot[i]
    similar_to_target = df_pivot.corrwith(target)
    corr_target = pd.DataFrame(similar_to_target, columns = ['PearsonR'])
    corr_target.dropna(inplace = True)
    corr_target = corr_target.sort_values('PearsonR', ascending = False)
    corr_target.index = corr_target.index.map(int)
    corr_target = corr_target.join(df)
    print(corr_target[:10].to_string(index=False))

In [54]:
recommend('Super Mario Bros.')

For movie (Super Mario Bros.)
- Top 10 movies recommended based on Pearsons'R correlation - 


c:\Users\jcrig\mlops\venv_mlops\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\jcrig\mlops\venv_mlops\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\jcrig\mlops\venv_mlops\Lib\site-packages\numpy\lib\function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\jcrig\mlops\venv_mlops\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\jcrig\mlops\venv_mlops\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


 PearsonR                                        title
      1.0                              Seven Angry Men
      1.0                 Miss Sweden (Fröken Sverige)
      1.0                                  Dragon Eyes
      1.0                              After Innocence
      1.0                                    Wave, The
      1.0                                  Lola Versus
      1.0                                     Highball
      1.0 3, 2, 1... Frankie Go Boom (Frankie Go Boom)
      1.0                                  Loggerheads
      1.0                              L'homme qui rit


In [77]:
def recommend(movie_title):
    print(f"For movie ({movie_title})")
    print("- Top 10 movies recommended based on Pearsons'R correlation - ")

    # Trouver l'index du film correspondant
    i = int(df.index[df['title'] == movie_title][0])

    # Corrélation de Pearson pour les ratings
    target = df_pivot[i]
    similar_to_target = df_pivot.corrwith(target)
    corr_target = pd.DataFrame(similar_to_target, columns=['PearsonR'])
    corr_target.dropna(inplace=True)
    corr_target = corr_target.sort_values('PearsonR', ascending=False)
    
    # Ajouter les informations sur les films
    corr_target.index = corr_target.index.map(int)
    corr_target = corr_target.join(df[['title']])

    # Calculer la similarité des genres
    def genre_similarity(row):
        # Genres du film cible sous forme binaire (0/1)
        target_genres = df.loc[i, df.columns[2:]]
        # Genres du film candidat sous forme binaire (0/1)
        candidate_genres = df.loc[row.name, df.columns[2:]]
        # Calculer le nombre de genres en commun
        return (target_genres & candidate_genres).sum()

    # Ajouter le score de similarité des genres
    corr_target['GenreSimilarity'] = corr_target.apply(genre_similarity, axis=1)

    # Trier par la corrélation de Pearson puis par la similarité des genres
    corr_target = corr_target.sort_values(['PearsonR', 'GenreSimilarity'], ascending=[False, False])

    # Afficher les 10 premiers films
    print(corr_target.head(10).to_string(index=False))

# Exemple d'utilisation
recommend('The Diary of Anne Frank')


For movie (The Diary of Anne Frank)
- Top 10 movies recommended based on Pearsons'R correlation - 


c:\Users\jcrig\mlops\venv_mlops\Lib\site-packages\numpy\lib\function_base.py:2889: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\jcrig\mlops\venv_mlops\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\jcrig\mlops\venv_mlops\Lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


ValueError: Cannot set a DataFrame with multiple columns to the single column GenreSimilarity